## Basic Clustering Algorithms

Unsupervised learning has many different flavors. Clustering is a large sub-field of unsupervised clustering. There are various clustering algorithms. We will cover a smaller subset. For all the methods within scikit-learn and their overview see this [link](https://scikit-learn.org/stable/modules/clustering.html).

Clustering algorithms mostly support the `fit` and `predict` functions as we will see below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import seaborn as sns; sns.set()

In [ ]:
# Let's create toy data

from sklearn.datasets import make_blobs
X, y_true = make_blobs(n_samples = 300, centers = 4,
                       cluster_std = 0.8)
plt.scatter(X[:, 0], X[:, 1], s = 20)
plt.show()

**K-Means**

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c = y_kmeans, s = 20, cmap = "viridis")

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c = "black", s = 200, alpha = 0.5)
plt.show()

Let's try it on the iris dataset

In [ ]:
from sklearn.datasets import load_iris

iris_data = load_iris()
X_iris = iris_data["data"]

# Use two dimensions to make visualization easier
X_iris = X_iris[:, [0,3]] 
y_iris = iris_data["target"]

# We know that there are 3 clusters
kmeans = KMeans(n_clusters = 3)
kmeans.fit(X_iris)
ykm = kmeans.predict(X_iris)

centers = kmeans.cluster_centers_

plt.figure(figsize=(16,6))
ax = plt.subplot(1,2,1)
#Data
ax.scatter(X_iris[:, 0], X_iris[:, 1], c = ykm, s = 50, cmap = "viridis")
#Large cluster Center
ax.scatter(centers[:, 0], centers[:, 1], c = "black", s = 200, alpha = 0.5)
ax.set_title('K-Means Cluster')

ax = plt.subplot(1,2,2)
ax.scatter(X_iris[:, 0], X_iris[:, 1], c = y_iris, s = 50, cmap = "viridis")
ax.set_title('Real Classes')

plt.show()

Let's see the classification score (taking the cluster membership as the predicted class). We are going to use the `adjusted_rand_score` for this.

In [ ]:
from sklearn.metrics import adjusted_rand_score
print('K-Means performance:',adjusted_rand_score(y_iris, ykm))
print('Accuracy of Random Prediction:',1/3)

**Number of Clusters**

We are supplying the number of clusters. In a realistic problem, we are more likely to not know this beforehand. There are multiple ways of picking number of clusters. We will see a simple idea called silhouette analysis, most suitable for K-Means. We calculate the silhouette score of a clustering outcome and pick the best one among multiple. Depending on the randomness of the initialization, we sometimes repeat the clustering with the same number of clusters as well.

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

def calcNumClustersWithSihouette(X):
    cluster_nums = np.arange(2,11)

    models = {}
    scores = {}

    for n_c in cluster_nums:
        model = KMeans(n_c)
        cluster_membership = model.fit_predict(X)
        scores[n_c] = silhouette_score(X, cluster_membership)
        models[n_c] = model

    np_scores = np.array(list(scores.values()))
    plt.plot(cluster_nums,np_scores)
    plt.gca().axvline(x=cluster_nums[np_scores.argmax()], color="k", linestyle="--")
    print(f'Highest Silhouette Performance is with {cluster_nums[np_scores.argmax()]} number of clusters and score {np_scores.max()}')
    
    return models[cluster_nums[np_scores.argmax()]]

In [ ]:
calcNumClustersWithSihouette(X)

In [ ]:
calcNumClustersWithSihouette(X_iris)

Silhouette scores are not always the best. More insight can be had by looking into silhouette plots. 

The below code is mainly taken from scikit-learn's [example](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html). 

In [ ]:
import matplotlib.cm as cm

range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()

**Gaussian Mixture Models (GMMs)**

In [ ]:
from sklearn.mixture import GaussianMixture
import matplotlib as mpl

gmm = GaussianMixture(n_components = 4)
gmm.fit(X)
y_gmm = gmm.predict(X)

y_gmm

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c = y_gmm, s = 20, cmap = "viridis")

centers = gmm.means_
plt.scatter(centers[:, 0], centers[:, 1], c = "black", s = 200, alpha = 0.5)
plt.show()

Let's also visualize the covariances

In [ ]:
gmm.covariances_

In [ ]:
def make_ellipses(gmm, ax, cmap):
    a = plt.get_cmap(cmap)
    colors = [a(1.*i/(gmm.n_components-1)) for i in range(gmm.n_components)]
    for n, color in enumerate(colors):
        if gmm.covariance_type == "full":
            covariances = gmm.covariances_[n][:2, :2]
        elif gmm.covariance_type == "tied":
            covariances = gmm.covariances_[:2, :2]
        elif gmm.covariance_type == "diag":
            covariances = np.diag(gmm.covariances_[n][:2])
        elif gmm.covariance_type == "spherical":
            covariances = np.eye(gmm.means_.shape[1]) * gmm.covariances_[n]
        v, w = np.linalg.eigh(covariances)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi # convert to degrees
        v = 3.0 * np.sqrt(2.0) * np.sqrt(v)
        ell = mpl.patches.Ellipse(gmm.means_[n, :2], v[0], v[1], 180 + angle, color = color)
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)
        ax.set_aspect("equal", "datalim")

In [ ]:
plt.figure(figsize=(8,8))
ax = plt.gca()
plt.scatter(X[:, 0], X[:, 1], c = y_gmm, s = 10, cmap = "viridis")
make_ellipses(gmm, ax, "viridis")
plt.show()

Since GMM is a distribution, we can get cluster membership probabilities

In [ ]:
gmm_probs = gmm.predict_proba(X) #argmax of columns is used to assign cluster membership
print(gmm_probs.shape)
print(gmm_probs[:10,:])

In [ ]:
# The iris Data set

# We know that there are 3 clusters
gmm = GaussianMixture(n_components = 3)
gmm.fit(X_iris)
ygmm = gmm.predict(X_iris)


plt.figure(figsize=(16,6))
ax = plt.subplot(1,2,1)
#Data
ax.scatter(X_iris[:, 0], X_iris[:, 1], c = ygmm, s = 50, cmap = "viridis")
make_ellipses(gmm, ax, "viridis")
ax.set_title('GMM Predictions')

ax = plt.subplot(1,2,2)
ax.scatter(X_iris[:, 0], X_iris[:, 1], c = y_iris, s = 50, cmap = "viridis")
ax.set_title('Real Classes')

plt.show()

In [ ]:
print('GMM performance:',adjusted_rand_score(y_iris, ygmm))
print('Accuracy of Random Prediction:',1/3)

**Agglomerative Clustering**

In [ ]:
from sklearn.cluster import AgglomerativeClustering

ac = AgglomerativeClustering(n_clusters = 3)
yac = ac.fit_predict(X_iris)
plt.scatter(X_iris[:, 0], X_iris[:, 1], c = yac, s = 50, cmap = "viridis")
plt.show()

print('Agglomerative Clustering Performance:',adjusted_rand_score(y_iris, yac))
print('Accuracy of Random Prediction:',1/3)

In [ ]:
ac2 = AgglomerativeClustering(n_clusters = 3, linkage = "average")
yac2 = ac2.fit_predict(X_iris)
plt.scatter(X_iris[:, 0], X_iris[:, 1], c = yac2, s = 50, cmap = "viridis")
plt.show()

print('Agglomerative Clustering Performance with Average Linkage:',adjusted_rand_score(y_iris, yac2))
print('Accuracy of Random Prediction:',1/3)

**Spectral Clustering**

In [ ]:
from sklearn.cluster import SpectralClustering
sc = SpectralClustering(3, n_init=100, assign_labels='discretize')
y_sc  = sc.fit_predict(X_iris)  

plt.scatter(X_iris[:, 0], X_iris[:, 1], c = y_sc, s = 50, cmap = "viridis")
plt.show()

print('Agglomerative Clustering Performance with Average Linkage:',adjusted_rand_score(y_iris, y_sc))
print('Accuracy of Random Prediction:',1/3)

**Digits**

Let's try these on the digits dataset

In [ ]:
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

digits = load_digits()
X = digits["data"]
y = digits["target"]
Xtsne = TSNE(n_components = 2).fit_transform(X)
km = KMeans(n_clusters = 10)
ykmeans = km.fit_predict(Xtsne)

print('TSNE + KMeans Performance:',adjusted_rand_score(y, ykmeans))

In [ ]:
def plotDigitProjections(Xin, y, title = None, show = True):
    plt.figure(figsize = (12, 8))
    for c in range(10):
        indexes = np.where(y == c)[0]
        x = Xin[indexes]
        plt.scatter(x[:, 0], x[:, 1], label = c)
    plt.xlabel("component 1")
    plt.ylabel("component 2")
    plt.legend()
    if title:
        plt.title(title)
    if(show):
        plt.show()

plotDigitProjections(Xtsne, ykmeans, "TSNE - KMeans Labels")
plotDigitProjections(Xtsne, y, "TSNE - True Labels")

In [ ]:
# What about PCA? Also to show that we can use unsupervised method in pipelines

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipe = Pipeline([("pca", PCA(n_components = 2)),
                 ("kmeans", KMeans(n_clusters = 10))])

pipe.fit(X)
y_pca_km = pipe.predict(X)
X_pca = pipe.named_steps["pca"].transform(X)

print('PCA + KMeans Performance:',adjusted_rand_score(y, y_pca_km))

In [ ]:
plotDigitProjections(X_pca, y_pca_km, "PCA - KMeans", True)
plotDigitProjections(X_pca, y, "PCA - True Labels")

In [ ]:
pipe = Pipeline([("pca", PCA(n_components = 40)),
                 ("kmeans", KMeans(n_clusters = 10))])

pipe.fit(X)
y_pca_km = pipe.predict(X)
X_pca = pipe.named_steps["pca"].transform(X)

print('PCA (40) + KMeans Performance:',adjusted_rand_score(y, y_pca_km))

In [ ]:
# What about no dimensionality reduction? - KMeans

kmeans_full = KMeans(n_clusters=10)
y_full = kmeans_full.fit_predict(X)

print('Full KMeans Performance:',adjusted_rand_score(y, y_full))

In [ ]:
# What about no dimensionality reduction? - GMM

kmeans_full = GaussianMixture(n_components=10)
y_full = kmeans_full.fit_predict(X)

print('Full GMM Performance:',adjusted_rand_score(y, y_full))

In [ ]:
from umap import UMAP

pipe_umap = Pipeline([("umap", UMAP(n_components = 2)),
                      ("kmeans", KMeans(n_clusters = 10))])

pipe_umap.fit(X)
y_umap_km = pipe_umap.predict(X)
X_umap = pipe_umap.named_steps["umap"].transform(X)

print('UMAP + KMeans Performance:',adjusted_rand_score(y, y_umap_km))

In [ ]:
plotDigitProjections(X_umap, y_umap_km, "UMAP - KMeans", True)
plotDigitProjections(X_umap, y, "UMAP - True Labels")